In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.DataFrame({'text':['people watch campusx','campusx watch campusx','people write comment','campusx write comment'],'output':[1,1,0,0]})
df

,text,output
0,people watch campusx,1
1,campusx watch campusx,1
2,people write comment,0
3,campusx write comment,0


**BAG Of WORDS TECHNIQUE**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [ ]:
bow=cv.fit_transform(df['text'])

In [ ]:
cv.vocabulary_

{'people': 2, 'watch': 3, 'campusx': 0, 'write': 4, 'comment': 1}

In [ ]:
print(bow[0].toarray())
print(bow[1].toarray())

[[1 0 1 1 0]]
[[2 0 0 1 0]]


In [ ]:
cv.transform(["campusx watch and write comment of campusx"]).toarray()

array([[2, 1, 0, 1, 1]])

n-grams

In [ ]:
df=pd.DataFrame({'text':['people watch campusx','campusx watch campusx','people write comment','campusx write comment'],'output':[1,1,0,0]})
df

,text,output
0,people watch campusx,1
1,campusx watch campusx,1
2,people write comment,0
3,campusx write comment,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(ngram_range=(2,3))

In [ ]:
bow=cv.fit_transform(df['text'])

In [ ]:
cv.vocabulary_

{'people watch': 4,
 'watch campusx': 8,
 'people watch campusx': 5,
 'campusx watch': 0,
 'campusx watch campusx': 1,
 'people write': 6,
 'write comment': 9,
 'people write comment': 7,
 'campusx write': 2,
 'campusx write comment': 3}

tf-idf techniques

In [ ]:
df=pd.DataFrame({'text':['people watch campusx','campusx watch campusx','people write comment','campusx write comment'],'output':[1,1,0,0]})
df

,text,output
0,people watch campusx,1
1,campusx watch campusx,1
2,people write comment,0
3,campusx write comment,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
tf.fit_transform(df['text']).toarray()

array([[0.49681612, 0.        , 0.61366674, 0.61366674, 0.        ],
       [0.8508161 , 0.        , 0.        , 0.52546357, 0.        ],
       [0.        , 0.57735027, 0.57735027, 0.        , 0.57735027],
       [0.49681612, 0.61366674, 0.        , 0.        , 0.61366674]])

In [ ]:
print(tf.idf_)
print(tf.get_feature_names_out())

[1.22314355 1.51082562 1.51082562 1.51082562 1.51082562]
['campusx' 'comment' 'people' 'watch' 'write']


assignment part

In [1]:
import numpy as np
import pandas as pd


In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vinay7410
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 938MB/s]

In [4]:
df=pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df['review'].count()

np.int64(50000)

In [7]:
df.isnull().sum()

,0
review,0
sentiment,0


preprocessing

In [8]:
# lowercase
df['review']=df['review'].str.lower()

In [9]:
# removing html tags
import re
def remove_html(raw_text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',raw_text)

In [10]:
df['review']=df['review'].apply(remove_html)

In [11]:
def remove_url(text):
  url=re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',text)

In [12]:
df['review']=df['review'].apply(remove_url)

In [13]:
# removing punctuation
import string
string.punctuation
exclude=string.punctuation

def remove_punc(text):
  return text.translate(str.maketrans('','',exclude))

In [14]:
df['review']=df['review'].apply(remove_punc)

In [15]:
# dictionary of chat words
chat_dict = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now'
}

# function to replace chat words
def replace_chat_words(raw_text):
    new_text = []
    for w in raw_text.split():
        if w.upper() in chat_dict:
            new_text.append(chat_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

# apply to DataFrame
df['review'] = df['review'].apply(replace_chat_words)


In [ ]:
# spell check
from textblob import TextBlob
df['review'] = df['review'].apply(lambda x: str(TextBlob(x).correct()))


KeyboardInterrupt: 

In [16]:
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.9 MB/s eta 0:00:00


In [17]:
# removing stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords.words('english')
def remove_stopwords(text):
  new_text=[]
  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)
  x=new_text[:]
  new_text.clear()
  return " ".join(x)
# df['review'][0]=df['review'][0].apply(remove_stopwords)
df.at[0, 'review'] = remove_stopwords(df.at[0, 'review'])
df.at[1, 'review'] = remove_stopwords(df.at[1, 'review'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
df['review'][1]
# before the stop word removal

' wonderful little production  filming technique   unassuming  oldtimebbc fashion  gives  comforting  sometimes discomforting sense  realism   entire piece  actors  extremely well chosen michael sheen    got   polari      voices  pat    truly see  seamless editing guided   references  williams diary entries     well worth  watching     terrificly written  performed piece  masterful production  one   great masters  comedy   life  realism really comes home   little things  fantasy   guard  rather  use  traditional dream techniques remains solid  disappears  plays   knowledge   senses particularly   scenes concerning orton  halliwell   sets particularly   flat  halliwells murals decorating every surface  terribly well done'

In [19]:
df['review'][1]
# after stop word removal\

' wonderful little production  filming technique   unassuming  oldtimebbc fashion  gives  comforting  sometimes discomforting sense  realism   entire piece  actors  extremely well chosen michael sheen    got   polari      voices  pat    truly see  seamless editing guided   references  williams diary entries     well worth  watching     terrificly written  performed piece  masterful production  one   great masters  comedy   life  realism really comes home   little things  fantasy   guard  rather  use  traditional dream techniques remains solid  disappears  plays   knowledge   senses particularly   scenes concerning orton  halliwell   sets particularly   flat  halliwells murals decorating every surface  terribly well done'

In [20]:
# Find out the number of words in the entire corpus and also the total number of unique words(vocabulary) using just python
words=0
for word in df['review']:
  words+=len(word.split())
print("corpus words",words)

unique_words = set()

for review in df['review']:
    unique_words.update(review.split())

print("unique words:", len(unique_words))


corpus words 11312520
unique words: 222581


In [21]:
# Problem 4

# Apply bag words and find the vocabulary also find the times each word has occured
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(token_pattern=r'\b[a-zA-Z]{2,}\b')
bow = cv.fit_transform(df['review'])



In [40]:
cv.vocabulary_

{'one': 132078,
 'reviewers': 155293,
 'mentioned': 116449,
 'watching': 203234,
 'oz': 135736,
 'episode': 58858,
 'youll': 210989,
 'hooked': 87399,
 'right': 156052,
 'exactly': 60778,
 'happened': 81632,
 'methe': 117046,
 'first': 67667,
 'thing': 187056,
 'struck': 178793,
 'brutality': 24123,
 'unflinching': 196530,
 'scenes': 161377,
 'violence': 200977,
 'set': 165615,
 'word': 208210,
 'go': 76195,
 'trust': 193489,
 'show': 167955,
 'faint': 63232,
 'hearted': 83239,
 'timid': 189456,
 'pulls': 148159,
 'punches': 148240,
 'regards': 152728,
 'drugs': 53230,
 'sex': 165875,
 'hardcore': 81923,
 'classic': 33266,
 'use': 198503,
 'wordit': 208229,
 'called': 26252,
 'nickname': 126859,
 'given': 75741,
 'oswald': 133896,
 'maximum': 114505,
 'security': 163526,
 'state': 176410,
 'penitentary': 138559,
 'focuses': 69052,
 'mainly': 111518,
 'emerald': 56774,
 'city': 32967,
 'experimental': 61873,
 'section': 163491,
 'prison': 146092,
 'cells': 29236,
 'glass': 75865,
 'fron

In [ ]:
# Problem 5

# Apply bag of bi-gram and bag of tri-gram and write down your observation about the dimensionality of the vocabulary

In [22]:
import re

def clean_text(text):
    text = re.sub(r'\d+', '', text)                      # remove numbers
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)           # shrink elongated words: "goooood" → "good"
    text = re.sub(r'[^a-zA-Z\s]', '', text)              # keep only alphabets + spaces
    return text.lower()

df['clean_review'] = df['review'].apply(clean_text)


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    ngram_range=(2,2),
    stop_words='english',
    token_pattern=r'\b[a-zA-Z]{3,}\b'
)

bow = cv.fit_transform(df['clean_review'])

print("Vocabulary size:", len(cv.get_feature_names_out()))
print("First 20 bigrams:", cv.get_feature_names_out()[:20])


Vocabulary size: 3115211
First 20 bigrams: ['aab tak' 'aachen months' 'aachen palm' 'aada adhura' 'aadha hamaari'
 'aadmittedly slim' 'aag actually' 'aag breaks' 'aag carried'
 'aag directors' 'aag does' 'aag doesnt' 'aag fail' 'aag figure'
 'aag going' 'aag hits' 'aag jugnu' 'aag make' 'aag ones' 'aag promote']


In [24]:
for n in [(1,1), (2,2), (3,3)]:
    cv = CountVectorizer(ngram_range=n,
                         stop_words='english',
                         token_pattern=r'\b[a-zA-Z]{3,}\b',
                         max_features=None)  # remove or set to cap size
    bow = cv.fit_transform(df['clean_review'])
    print(f"{n}-gram vocabulary size:", len(cv.get_feature_names_out()))


(1, 1)-gram vocabulary size: 213139
(2, 2)-gram vocabulary size: 3115211
(3, 3)-gram vocabulary size: 4848252


In [ ]:
# Problem 6

# Apply tf-idf and find out the idf scores of words, also find out the vocabulary.


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
tf.fit_transform(df['review']).toarray()

NameError: name 'df' is not defined

In [ ]:
print(tf.idf_)
print(tf.get_feature_names_out())